In [1]:
from __future__ import print_function
from bs4 import BeautifulSoup
from dateutil import parser
import requests
import json
import time
import pandas as pd

In [2]:
USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

In [3]:
keywords = ['python','rstudio','data science','data analytics','business analytics','big data']  

In [4]:
# Google Search Engine

def fetch_results(search_term, number_results, language_code, country_code, time_results):
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'
    escaped_search_term = search_term.replace(' ', '+')

    google_url = 'https://www.google.es/search?q={}&num={}&hl={}&cr=country{}&as_qdr={}'.format(escaped_search_term, 
                                                                                                 number_results,
                                                                                                 language_code,
                                                                                                 country_code,
                                                                                                 time_results)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()

    return search_term, response.text

In [5]:
# Parse Results for Google Search
def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')

    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    for result in result_block:

        link = result.find('a', href=True)
        title = result.find('h3')
        description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'rank': rank, 'title': title, 
                                      'description': description, 'link':link})
                rank += 1
    return found_results

In [6]:
def on_data(data):
    #This is the meat of the script...it connects to your mongoDB and stores the search results
    try:
        # Create dataJson
        for i in data:
                   
        #grab the keys and values
            keyword = i['keyword']
            rank = i['rank']
            title = i['title']
            description = i['description']
            link = i['link']
           
            #insert the data into the MySQL database
            store_data(keyword, rank, title, description, link)

    except Exception as e:
        print(e)

In [7]:
def store_data(keyword, rank, title, description, link):
    google_se = pd.DataFrame({
        "keyword":keyword,
        "rank":rank,
        "title":title,
        "description":description,
        "link":link   
    })
    return google_se

In [8]:
# Scraping Web function for Google SEARCH only 50 results, spanish, Spain, last 1 day
def scrape_web(search_term):
        keyword, html = fetch_results(search_term, 50, "es", "ES" ,"d") # last 1 day
        results = parse_results(html, keyword)
        return results

In [22]:
# # Scraping Web function for Google SEARCH only 1000 results, spanish, Spain, all results
# def scrape_web(search_term):
#         keyword, html = fetch_results(search_term, 1000, "es", "ES" ,"all") # all results
#         results = parse_results(html, keyword)
#         return results

In [9]:
# Testing googleScraping
data = []
#results = scrape_web(keywords[0])
results = scrape_web('master data science')
for result in results:
    data.append(result)

In [10]:
data

[{'keyword': 'master data science',
  'rank': 1,
  'title': 'Máster en Data Strategy & Analytics del centro MIOTI. Tech ...',
  'description': None,
  'link': 'https://www.aprendemas.com/es/curso/master-en-data-strategy-analytics-1180954/?utm_source=contenidos&utm_medium=contenidos&utm_campaign=contenidos-mioti'},
 {'keyword': 'master data science',
  'rank': 2,
  'title': 'Master Classes Business Analytics y la realidad empresarial',
  'description': None,
  'link': 'https://www.villanueva.edu/master-classes-business-analytics-y-la-realidad-empresarial/'},
 {'keyword': 'master data science',
  'rank': 3,
  'title': 'Másteres Online - UCO - Universidad de Córdoba',
  'description': None,
  'link': 'https://online.uco.es/landings-latam/conoce-nuestros-masters-latam/?utm_source=ADS&utm_medium=CPC&utm_campaign=15289479830&utm_content=562095905037&utm_term=universidad%20de%20c%C3%B3rdoba&gclid=EAIaIQobChMIubGU-7yu9AIVled3Ch0FLAPpEAAYASAAEgKpZvD_BwE'},
 {'keyword': 'master data science',
  

In [25]:
dataset = pd.DataFrame(data)

In [26]:
dataset

,keyword,rank,title,description,link
0,master data science,1,Máster en Big Data Science - ETSIT,El Máster en Big Data Science de la Universida...,https://www.tel.uva.es/docencia/planes/masterb...
1,master data science,2,STER UNIVERSITARIO EN CIENCIA DE DATOS / MASTE...,TRABAJO DE FIN DE MÁSTER. 6 ECTS. Desde FUNDAM...,http://www.uimp.es/postgrado/estudios/fichaest...
2,master data science,3,MASTER DATA SCIENCE: Master Data Science: Big ...,Doble Titulación: - Titulación de Master Data ...,https://www.mastermania.com/master-data-scienc...
3,master data science,4,Trabajos de Master data science - julio 2019 |...,226 empleos de Master data science disponibles...,https://www.indeed.es/Ofertas-de-Master-data-s...
4,master data science,5,Máster en Big Data Analytics - UPV Universitat...,La UPV ofrece la oportunidad a profesionales y...,https://bigdata.inf.upv.es/
5,master data science,6,Master Data Science - FSSM - Publicaciones | F...,Master Data Science - FSSM. 521 Me gusta · 94 ...,https://es-es.facebook.com/pages/category/Scie...
6,master data science,7,"Master de 10000 € en Data Science, ¿merece la ...","Hola shurs, Me ha quedado una asignatura para ...",https://www.forocoches.com/foro/showthread.php...
7,master data science,8,¿Por qué el analista de datos es uno de los pe...,"12 feb. 2018 - Noticias, Analytics: El salario...",https://www.computing.es/analytics/noticias/11...


In [11]:
on_data(data)

If using all scalar values, you must pass an index


In [ ]:
if __name__ == '__main__':
    
    data = []
    for keyword in keywords:
        try:
            results = scrape_web(keyword)
            
            for result in results:
                data.append(result)
             
        except Exception as e:
            print(e)
        finally:
            time.sleep(60)
    #print(data)
    # Call function to store data
    on_data(data)

## Sistrix